In [1]:
import h5py
import numpy as np
import pandas as pd
import json
import yaml
from ludwig import LudwigModel
import copy
import ray
from ludwig.utils.misc import merge_dict
import re

/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
pattern = "^{{.*}}$"
list_index_pattern = "^\[\d+\]$"


def get_keys_from_list(l, path=""):
    parameters = []

    for index, v in enumerate(l):
        if isinstance(l[index], str):
             if re.match(pattern, l[index], flags=0) is not None:
                parameters.append([path + "[" + index + "]", v])

        elif isinstance(l[index], dict):
            p = get_keys_from_dict(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)
        elif isinstance(l[index], list):
            p = get_keys_from_list(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)

    return parameters


def get_keys_from_dict(dct, path=""):
    parameters = []
    for k, v in dct.items():
        if isinstance(dct[k], str):
            if re.match(pattern, dct[k], flags=0) is not None:
                parameters.append([path + k,dct[k]])

        elif isinstance(dct[k], dict):
            p = get_keys_from_dict(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)

        elif isinstance(dct[k], list):
            p = get_keys_from_list(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)

    return parameters



def update_param(dct, path, value):
    if len(path) == 1:
        dct[path[0]] = value
    else:
        l = re.match(list_index_pattern, path[0], flags=0)
        if l is not None and isinstance(dct, list):
            index = int(l.group(0)[1:-1])
            update_param(dct[index], path[1:], value)
        else:
             update_param(dct[path[0]], path[1:], value)


def build_model(base_model, config):
    for k, v in config.items():
        p = k.split('->')
        update_param(base_model, p, v)
    return base_model

In [3]:
from ray.tune import grid_search

def build_search_space(annotated_model):
    pattern = "^{{(.*)}}$"
    grid_search_space = {}
    keys = get_keys_from_dict(annotated_model)
    for a in keys:
        grid_search_space[a[0]]= grid_search(yaml.load(re.match(pattern, a[1], flags=0)[1]))

    return grid_search_space


In [15]:
#use full path for Ray
data = '/Users/bm255022/Projects/Teradata/ludwig/examples/heart/heart.hdf5'
metadata = '/Users/bm255022/Projects/Teradata/ludwig/examples/heart/heart.json'

def train(base, config, reporter):
    new_model_def = build_model(base, config)
    model = LudwigModel(new_model_def)
    train_stats = model.train(data_hdf5=data, train_set_metadata_json=metadata)
    return reporter(mean_accuracy=np.sort(train_stats['test']['narrowing-diagnosis']['accuracy'])[-1], done=True)


In [21]:
from ray.tune import register_trainable, grid_search, run_experiments

ray.shutdown()
ray.init()

with open("model.yaml", 'r') as stream:
    annotated_model = yaml.load(stream)

grid_search_space = build_search_space(annotated_model)
register_trainable('train', lambda cfg, rptr: train(annotated_model, cfg, rptr))
run_experiments({'heart': {
    'run': 'train',
    'stop': {'mean_accuracy': 0.9},
    'config': grid_search_space}
    })

2019-04-01 18:33:36,392	WARNING worker.py:1381 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-04-01 18:33:36,396	INFO node.py:439 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-01_18-33-36_37428/logs.
2019-04-01 18:33:36,508	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:30760 to respond...
2019-04-01 18:33:36,636	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:61348 to respond...
2019-04-01 18:33:36,640	INFO services.py:761 -- Starting Redis shard with 3.44 GB max memory.
2019-04-01 18:33:36,670	INFO services.py:1449 -- Starting the Plasma object store with 5.15 GB memory using /tmp.
/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please re

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/8 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)
Result logdir: /Users/bm255022/ray_results/heart
Number of trials: 12 ({'RUNNING': 1, 'PENDING': 11})
PENDING trials:
 - train_1_combiner->fc_size=128,combiner->num_fc_layers=1,training->learning_rate=0.001:	PENDING
 - train_2_combiner->fc_size=64,combiner->num_fc_layers=2,training->learning_rate=0.001:	PENDING
 - train_3_combiner->fc_size=128,combiner->num_fc_layers=2,training->learning_rate=0.001:	PENDING
 - train_4_combiner->fc_size=64,combiner->num_fc_layers=1,training->learning_rate=0.005:	PENDING
 - train_5_combiner->fc_size=128,combiner->num_fc_layers=1,training->learning_rate=0.005:	PENDING
 - train_6_combiner->fc_size=64,combiner->num_fc_layers=2

(pid=50465) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=50461) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=50475) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
(pid=50475)   return f(*args, **kwds)
(pid=50476) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
(pid=50476)   return f(*args, **kwds)
Result for train_4_combiner->fc_size=64,combiner->num_fc_layers=1,training->learning_rate=0.

(pid=50469) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
(pid=50469)   "Use dataset[()] instead.", H5pyDeprecationWarning)
(pid=50474) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
(pid=50474)   "Use dataset[()] instead.", H5pyDeprecationWarning)
(pid=50475) 2019-04-01 18:34:33.079945: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=50476) 2019-04-01 18:34:33.075583: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=50469) 2019-04-01 18:34:33.912550: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supp

[train_0_combiner->fc_size=64,combiner->num_fc_layers=1,training->learning_rate=0.001,
 train_1_combiner->fc_size=128,combiner->num_fc_layers=1,training->learning_rate=0.001,
 train_2_combiner->fc_size=64,combiner->num_fc_layers=2,training->learning_rate=0.001,
 train_3_combiner->fc_size=128,combiner->num_fc_layers=2,training->learning_rate=0.001,
 train_4_combiner->fc_size=64,combiner->num_fc_layers=1,training->learning_rate=0.005,
 train_5_combiner->fc_size=128,combiner->num_fc_layers=1,training->learning_rate=0.005,
 train_6_combiner->fc_size=64,combiner->num_fc_layers=2,training->learning_rate=0.005,
 train_7_combiner->fc_size=128,combiner->num_fc_layers=2,training->learning_rate=0.005,
 train_8_combiner->fc_size=64,combiner->num_fc_layers=1,training->learning_rate=0.0005,
 train_9_combiner->fc_size=128,combiner->num_fc_layers=1,training->learning_rate=0.0005,
 train_10_combiner->fc_size=64,combiner->num_fc_layers=2,training->learning_rate=0.0005,
 train_11_combiner->fc_size=128,co

(pid=50469) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.


In [20]:
rm -rf ~/ray_results/heart
